In [ ]:
import pandas as pd
import numpy as np
import json
import re
import warnings
warnings.filterwarnings("ignore")
                        
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [10]:
import os
print(os.listdir())

['.anaconda', '.conda', '.condarc', '.config', '.continuum', '.gitconfig', '.idlerc', '.ipynb_checkpoints', '.ipython', '.jupyter', '.matplotlib', '.ms-ad', '.ssh', '.streamlit', '.VirtualBox', '.vscode', '3D Objects', 'anaconda3', 'AppData', 'Application Data', 'assignment.ipynb', 'Contacts', 'Cookies', 'Desktop', 'Documents', 'Downloads', 'Favorites', 'IntelGraphicsProfiles', 'Links', 'Local Settings', 'ML Assignment.ipynb', 'Music', 'My Documents', 'myenv', 'NetHood', 'news-sentiment-app', 'news_audio.mp3', 'new_env', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM.blf', 'NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000001.regtrans-ms', 'NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000002.regtrans-ms', 'ntuser.ini', 'OneDrive', 'output.mp3', 'Pictures', 'PrintHood', 'Recent', 'Saved Games', 'Searches', 'SendTo', 'speech.mp3', 'Start Menu', 'Templates', 'test.mp3', 'Unti

In [3]:
import json

with open("C:/Users/admin/Desktop/New Assignment/algoparams_clean.json", "r") as f:
    config = json.load(f)

design_data = config["design_state_data"]
print(" JSON Loaded Successfully")
print(" Target Column:", design_data["target"]["target"])
print(" Prediction Type:", design_data["target"]["prediction_type"])

 JSON Loaded Successfully
 Target Column: petal_width
 Prediction Type: Regression


In [4]:
import pandas as pd

df = pd.read_csv("C:/Users/admin/Desktop/New Assignment/iris.csv")
print("✅ Dataset loaded successfully:")
df.head()

✅ Dataset loaded successfully:


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris(as_frame=True)
df = iris.frame
df["petal_width"] = df["target"]  
df["species"] = df["target"].map({0: "setosa", 1: "versicolor", 2: "virginica"})
df.drop(columns=["target"], inplace=True)

print("✅ Iris dataset loaded from sklearn:")
df.head()

✅ Iris dataset loaded from sklearn:


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),petal_width,species
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [6]:
print(df.columns.tolist())

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)', 'petal_width', 'species']


In [7]:
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', ...]

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', Ellipsis]

In [8]:
df.rename(columns={
    "sepal length (cm)": "sepal_length",
    "sepal width (cm)": "sepal_width",
    "petal length (cm)": "petal_length",
    "petal width (cm)": "petal_width"
}, inplace=True)

In [9]:
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
df = iris.frame
df["petal_width"] = df["target"]
df["species"] = df["target"].map({0: "setosa", 1: "versicolor", 2: "virginica"})
df.drop(columns=["target"], inplace=True)

df.rename(columns={
    "sepal length (cm)": "sepal_length",
    "sepal width (cm)": "sepal_width",
    "petal length (cm)": "petal_length",
    "petal width (cm)": "petal_width"
}, inplace=True)

print("✅ Dataset reset to original state.")

✅ Dataset reset to original state.


In [42]:
selected_features = []
feature_handling = design_data["feature_handling"]

for feature_name, meta in feature_handling.items():
    if meta["is_selected"]:
        selected_features.append(feature_name)

df = df[selected_features]

for feature_name in selected_features:
    details = feature_handling[feature_name]["feature_details"]
    
    if "missing_values" in details and details["missing_values"] == "Impute":
        impute_type = details["impute_with"]
        impute_value = details["impute_value"]

        if impute_type == "Average of values":
            avg_val = df[feature_name].mean()
            df[feature_name] = df[feature_name].fillna(avg_val)

        elif impute_type == "custom":
           df[feature_name] = df[feature_name].fillna(impute_value)


print("✅ Feature selection & missing value handling complete.")
df.head()

✅ Feature selection & missing value handling complete.


,sepal_length,sepal_width,petal_length,petal_width,petal_width,species
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [17]:
selected_features = design_data.get("selected_features", [])
feature_handling = design_data.get("feature_handling", {})

for feature_name in selected_features:
    details = feature_handling.get(feature_name, {}).get("feature_details", {})

    if details.get("missing_values") == "Impute":
        print(f"{feature_name}: Imputation required")

        impute_type = details.get("impute_with")
        impute_value = details.get("impute_value")

        if impute_type == "Average of values":
            avg_val = df[feature_name].mean()
            df[feature_name] = df[feature_name].fillna(avg_val)

        elif impute_type == "custom" and impute_value is not None:
            df[feature_name] = df[feature_name].fillna(impute_value)

In [11]:
feature_gen = design_data["feature_generation"]

for pair in feature_gen.get("linear_interactions", []):
    col1, col2 = pair
    new_col = f"{col1}_x_{col2}"
    df[new_col] = df[col1] * df[col2]

for interaction in feature_gen.get("polynomial_interactions", []):
    try:
        col1, col2 = interaction.split("/")
        new_col = f"{col1}_div_{col2}"
        df[new_col] = df[col1] / df[col2]
    except:
        print(f" Skipping invalid polynomial interaction: {interaction}")

for interaction in feature_gen.get("explicit_pairwise_interactions", []):
    try:
        col1, col2 = interaction.split("/")
        new_col = f"{col1}_pair_{col2}"
        df[new_col] = df[col1] * df[col2]
    except:
        print(f" Skipping invalid pairwise interaction: {interaction}")

print("✅ Feature generation complete.")
df.head()

 Skipping invalid polynomial interaction: petal_width/species
 Skipping invalid pairwise interaction: petal_width/sepal_length
✅ Feature generation complete.


,sepal_length,sepal_width,petal_length,petal_width,petal_width,species,petal_length_x_sepal_width,petal_length_div_sepal_width,sepal_width_pair_sepal_length
0,5.1,3.5,1.4,0.2,0,setosa,4.90,0.400000,17.85
1,4.9,3.0,1.4,0.2,0,setosa,4.20,0.466667,14.70
2,4.7,3.2,1.3,0.2,0,setosa,4.16,0.406250,15.04
3,4.6,3.1,1.5,0.2,0,setosa,4.65,0.483871,14.26
4,5.0,3.6,1.4,0.2,0,setosa,5.04,0.388889,18.00


In [35]:
from sklearn.ensemble import RandomForestRegressor

target_col = design_data["target"]["target"]

X = df.drop(columns=[target_col], errors='ignore').select_dtypes(include=['number'])
y = df[target_col]

if target_col not in df.columns:
    print(f" Target column '{target_col}' not found in dataframe. Skipping reduction.")
else:
    model = RandomForestRegressor(n_estimators=5, max_depth=6, random_state=0)
    model.fit(X, y)

    importances = model.feature_importances_
    feature_scores = pd.Series(importances, index=X.columns)
    top_features = feature_scores.nlargest(int(design_data["feature_reduction"]["num_of_features_to_keep"])).index.tolist()

    df = df[top_features + [target_col]]
    print("✅ Feature reduction complete. Selected features:", top_features)

df.head()

✅ Feature reduction complete. Selected features: ['petal_length_x_sepal_width', 'petal_length', 'petal_length_div_sepal_width', 'sepal_width_pair_sepal_length']


,petal_length_x_sepal_width,petal_length,petal_length_div_sepal_width,sepal_width_pair_sepal_length,petal_width,petal_width
0,4.90,1.4,0.400000,17.85,0.2,0
1,4.20,1.4,0.466667,14.70,0.2,0
2,4.16,1.3,0.406250,15.04,0.2,0
3,4.65,1.5,0.483871,14.26,0.2,0
4,5.04,1.4,0.388889,18.00,0.2,0


In [10]:
target_col = "petal_width"
X = df.drop(columns=[target_col]) 
y = df[target_col]

from sklearn.preprocessing import LabelEncoder
for col in X.columns:
    if X[col].dtype == 'object':
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

grid_model = GridSearchCV(RandomForestRegressor(), param_grid, cv=5, scoring='r2')
grid_model.fit(X_train, y_train)

print("Best Params:", grid_model.best_params_)
y_pred = grid_model.predict(X_test)
print("R² Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))

Best Params: {'max_depth': 4, 'n_estimators': 25}
R² Score: 0.9467463305048822
MAE: 0.15113512161948525


In [4]:
import numpy as np
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(" RMSE:", rmse)

 RMSE: 0.1881843196665673
